In [1]:
cd /content/drive/MyDrive/Data Mining

/content/drive/MyDrive/Data Mining


In [3]:
!pip install pytrends

In [68]:
import os
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from pytrends import dailydata
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [69]:
bitcoin_1_day_data = pd.read_csv('BTC-USDT-1d.csv')

In [70]:
bitcoin_1_day_data.drop(columns = ['quote_asset_volume','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore','close_time'],inplace=True)
#bitcoin_1_day_data.drop(columns = ["volume","number_of_trades"],inplace=True)

In [71]:
bitcoin_1_day_data.head()

,open_time,open,high,low,close,volume,number_of_trades
0,1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377,3427
1,1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264,5233
2,1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763,2153
3,1503187200000,4120.98,4211.08,4032.62,4086.29,467.083022,2321
4,1503273600000,4069.13,4119.62,3911.79,4016.00,691.743060,3972


In [72]:
bitcoin_1_day_data['open_time'] = pd.to_datetime(bitcoin_1_day_data['open_time'],unit='ms').astype('str')


In [73]:
bitcoin_1_day_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
open,1571.0,16864.839064,16919.821554,3188.010000,6709.100000,9231.400000,1.641526e+04,6.752582e+04
high,1571.0,17379.047804,17422.350672,3276.500000,6836.015000,9400.000000,1.715969e+04,6.900000e+04
low,1571.0,16277.967537,16334.396860,2817.000000,6545.305000,9035.000000,1.572236e+04,6.622240e+04
close,1571.0,16892.701222,16935.015374,3189.020000,6712.280000,9232.000000,1.653847e+04,6.752583e+04
volume,1571.0,50599.094700,36573.812596,228.108068,28436.402899,43648.036943,6.503041e+04,4.022017e+05
number_of_trades,1571.0,749228.818587,761051.367406,2153.000000,211014.000000,430359.000000,1.096416e+06,6.331062e+06


In [74]:
'''
google_trends_list = ['bitcoin chart',
 'bitcoin dollar',
 'bitcoin dolar',
 'bitcoin news',
 'bitcoin mining',
 'btc',
 'bitcoin usd',
 'bitcoin',
 'bitcoin to usd',
 'bitcoin price usd',
 'bitcoin stock',
 'bitcoin wallet',
 'bitcoin value',
 'cryptocurrency',
 'coinbase',
 'miner',
 'price bitcoin',
 'sell bitcoin',
 'sell btc',
 'what is bitcoin',
 'buy bitcoin']
# download daily trends data
from pytrends import dailydata
for topic in google_trends_list:
  print(topic)
  dailydata.get_daily_data(topic, 2017, 8, 2021, 12, geo = '').to_csv("google trends data/"+topic+'.csv')
'''



'\ngoogle_trends_list = [\'bitcoin chart\',\n \'bitcoin dollar\',\n \'bitcoin dolar\',\n \'bitcoin news\',\n \'bitcoin mining\',\n \'btc\',\n \'bitcoin usd\',\n \'bitcoin\',\n \'bitcoin to usd\',\n \'bitcoin price usd\',\n \'bitcoin stock\',\n \'bitcoin wallet\',\n \'bitcoin value\',\n \'cryptocurrency\',\n \'coinbase\',\n \'miner\',\n \'price bitcoin\',\n \'sell bitcoin\',\n \'sell btc\',\n \'what is bitcoin\',\n \'buy bitcoin\']\n# download daily trends data\nfrom pytrends import dailydata\nfor topic in google_trends_list:\n  print(topic)\n  dailydata.get_daily_data(topic, 2017, 8, 2021, 12, geo = \'\').to_csv("google trends data/"+topic+\'.csv\')\n'

In [75]:
google_trends_dataframe = pd.DataFrame()
google_trends_list = ['buy bitcoin','sell bitcoin']
for keyword in google_trends_list:
  if not len(google_trends_dataframe):
    google_trends_dataframe = pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]]
  if len(google_trends_dataframe):
    google_trends_dataframe = pd.merge(google_trends_dataframe,pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]])
merged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on='open_time',right_on='date',how='outer')
merged_dataframe.drop(columns=['date'],inplace=True)

In [76]:
'''
#to download the google trends hourly data
google_trends_list = ['buy bitcoin','sell bitcoin']
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

pytrends.get_historical_interest(google_trends_list, year_start=2017,month_start=8, 
                                 day_start=1,hour_start=0, year_end=2017,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2017.csv')                       
pytrends.get_historical_interest(google_trends_list, year_start=2018,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2018,month_end=12,day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2018.csv')
pytrends.get_historical_interest(google_trends_list, year_start=2019,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2019,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2019.csv')
pytrends.get_historical_interest(google_trends_list, year_start=2020,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2020,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2020.csv')

pytrends.get_historical_interest(google_trends_list, year_start=2021,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2021,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2021.csv')                                                                                                                                    
'''                      

"\n#to download the google trends hourly data\ngoogle_trends_list = ['buy bitcoin','sell bitcoin']\nfrom pytrends.request import TrendReq\n\npytrends = TrendReq(hl='en-US', tz=360)\n\npytrends.get_historical_interest(google_trends_list, year_start=2017,month_start=8, \n                                 day_start=1,hour_start=0, year_end=2017,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2017.csv')                       \npytrends.get_historical_interest(google_trends_list, year_start=2018,month_start=1, \n                                 day_start=1,hour_start=0, year_end=2018,month_end=12,day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2018.csv')\npytrends.get_historical_interest(google_trends_list, year_start=2019,month_start=1, \n                                 day_start=1,hour_start=0, year_end=2019,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_d

In [77]:
'''
google_trends_dataframe = []
for file in os.listdir():
  if file[:19] == "hourly_trends_data_":
    if len(google_trends_dataframe)==0:
      google_trends_dataframe.append(pd.read_csv(file))
    else:
      google_trends_dataframe.append(pd.read_csv(file))
google_trends_dataframe = pd.concat(google_trends_dataframe)
google_trends_dataframe.dropna(inplace=True)
google_trends_dataframe.drop(columns=['isPartial'],inplace=True)
bitcoin_1_day_data['open_time'] = bitcoin_1_day_data['open_time'].apply(lambda x:x[:-4])
merged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on='open_time',right_on='date',how='inner')
merged_dataframe.drop(columns=['date'],inplace=True)
'''

'\ngoogle_trends_dataframe = []\nfor file in os.listdir():\n  if file[:19] == "hourly_trends_data_":\n    if len(google_trends_dataframe)==0:\n      google_trends_dataframe.append(pd.read_csv(file))\n    else:\n      google_trends_dataframe.append(pd.read_csv(file))\ngoogle_trends_dataframe = pd.concat(google_trends_dataframe)\ngoogle_trends_dataframe.dropna(inplace=True)\ngoogle_trends_dataframe.drop(columns=[\'isPartial\'],inplace=True)\nbitcoin_1_day_data[\'open_time\'] = bitcoin_1_day_data[\'open_time\'].apply(lambda x:x[:-4])\nmerged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on=\'open_time\',right_on=\'date\',how=\'inner\')\nmerged_dataframe.drop(columns=[\'date\'],inplace=True)\n'

In [78]:
merged_dataframe.dropna(inplace=True)

In [79]:
merged_dataframe['close']=merged_dataframe['close']>merged_dataframe['open']

In [80]:
merged_dataframe.head()

,open_time,open,high,low,close,volume,number_of_trades,buy bitcoin,sell bitcoin
0,2017-08-17,4261.48,4485.39,4200.74,True,795.150377,3427.0,12.72,10.44
1,2017-08-18,4285.08,4371.52,3938.77,False,1199.888264,5233.0,18.24,6.12
2,2017-08-19,4108.37,4184.69,3850.00,True,381.309763,2153.0,16.08,13.68
3,2017-08-20,4120.98,4211.08,4032.62,False,467.083022,2321.0,10.40,10.92
4,2017-08-21,4069.13,4119.62,3911.79,False,691.743060,3972.0,8.32,7.28


In [81]:
n = len(merged_dataframe)
training_set = merged_dataframe.iloc[0:int(n*0.7),1:].values
test_set = merged_dataframe.iloc[int(n*0.7):,1:].values
num_features = training_set.shape[1]

In [82]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
print(sc.fit(training_set))
training_set_scaled = sc.transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i,:])
    y_train.append(training_set_scaled[i,3])
X_train, y_train = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)

MinMaxScaler()


In [83]:
test_set_scaled = sc.transform(test_set)
X_test = []
y_test = []
for i in range(60, len(test_set)):
    X_test.append(test_set_scaled[i-60:i, :])
    y_test.append(test_set_scaled[i,3])
X_test,y_test = tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)

In [89]:
# Compiling the RNN linear
def compile_and_fit(model, X_train, y_train):
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[callback])
  return history


def compile_and_fit_log(model, X_train, y_train):
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  '''
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  '''
  model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
  
  history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[callback])
  return history

In [91]:
google_trends_model = Sequential()
google_trends_model.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], num_features)))
google_trends_model.add(Dropout(0.2))
google_trends_model.add(LSTM(units = 120, return_sequences = True))
google_trends_model.add(Dropout(0.2))
google_trends_model.add(LSTM(units = 60))
google_trends_model.add(Dropout(0.2))
'''
google_trends_model.add(Dense(units = 1))
google_trends_model.add(Activation('linear'))
history = compile_and_fit(google_trends_model,X_train, y_train)
google_trends_model.save('my_model_with_1d_google_trends_price_prediction')
'''
google_trends_model.add(Dense(units = 1))
history = compile_and_fit_log(google_trends_model,X_train, y_train)
google_trends_model.save('my_model_with_1d_google_trends_price_change')
model = Sequential()
model.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], num_features-2)))
model.add(Dropout(0.2))
model.add(LSTM(units = 120, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 60))
model.add(Dropout(0.2))
'''
model.add(Dense(units = 1))
model.add(Activation('linear'))
history = compile_and_fit(model,X_train[:,:,:-2], y_train)
model.save('my_model_with_1d_without_google_trends_price_prediction')
'''
model.add(Dense(units = 1))
history = compile_and_fit_log(model,X_train[:,:,:-2], y_train)
model.save('my_model_with_1d_without_google_trends_price_change')

Epoch 1/100
33/33 [==============================] - 14s 220ms/step - loss: 0.6994 - accuracy: 0.4730
Epoch 2/100
33/33 [==============================] - 7s 206ms/step - loss: 0.6931 - accuracy: 0.4740
Epoch 3/100
33/33 [==============================] - 7s 209ms/step - loss: 0.6926 - accuracy: 0.4740
Epoch 4/100
33/33 [==============================] - 7s 211ms/step - loss: 0.6925 - accuracy: 0.4740
Epoch 5/100
33/33 [==============================] - 7s 213ms/step - loss: 0.6919 - accuracy: 0.4740
Epoch 6/100
33/33 [==============================] - 7s 214ms/step - loss: 0.6915 - accuracy: 0.4740
Epoch 7/100
33/33 [==============================] - 7s 211ms/step - loss: 0.6930 - accuracy: 0.4740
Epoch 8/100
33/33 [==============================] - 7s 220ms/step - loss: 0.6916 - accuracy: 0.4740
Epoch 9/100
33/33 [==============================] - 7s 206ms/step - loss: 0.6919 - accuracy: 0.4740


INFO:tensorflow:Assets written to: my_model_with_1d_google_trends_price_change/assets


INFO:tensorflow:Assets written to: my_model_with_1d_google_trends_price_change/assets


Epoch 1/100
33/33 [==============================] - 15s 226ms/step - loss: 0.6957 - accuracy: 0.4740
Epoch 2/100
33/33 [==============================] - 7s 215ms/step - loss: 0.6936 - accuracy: 0.4740
Epoch 3/100
33/33 [==============================] - 7s 204ms/step - loss: 0.6927 - accuracy: 0.4740
Epoch 4/100
33/33 [==============================] - 7s 209ms/step - loss: 0.6926 - accuracy: 0.4740
Epoch 5/100
33/33 [==============================] - 7s 212ms/step - loss: 0.6926 - accuracy: 0.4740
Epoch 6/100
33/33 [==============================] - 7s 200ms/step - loss: 0.6925 - accuracy: 0.4740
Epoch 7/100
33/33 [==============================] - 7s 205ms/step - loss: 0.6926 - accuracy: 0.4740
Epoch 8/100
33/33 [==============================] - 7s 217ms/step - loss: 0.6912 - accuracy: 0.4740
Epoch 9/100
33/33 [==============================] - 7s 213ms/step - loss: 0.6926 - accuracy: 0.4740
Epoch 10/100
33/33 [==============================] - 7s 208ms/step - loss: 0.6919 - accur

INFO:tensorflow:Assets written to: my_model_with_1d_without_google_trends_price_change/assets


INFO:tensorflow:Assets written to: my_model_with_1d_without_google_trends_price_change/assets


In [92]:
#logistic
test_loss, test_acc = model.evaluate(X_test[:,:,:-2],  y_test, verbose=2)
test_loss, test_acc = google_trends_model.evaluate(X_test,  y_test, verbose=2)

13/13 - 2s - loss: 0.7030 - accuracy: 0.4647 - 2s/epoch - 168ms/step
13/13 - 2s - loss: 0.6964 - accuracy: 0.4647 - 2s/epoch - 165ms/step


In [24]:
'''
#linear
predicted_stock_price = model.predict(X_test[:,:,:-2])/sc.scale_[3]
predicted_stock_price_with_google_trends = google_trends_model.predict(X_test)/sc.scale_[3]
# Visualising the results
plt.figure(figsize=(18,16))
plt.plot(merged_dataframe.iloc[int(n*0.7)+60:,4].values, color = 'red', label = 'Real Bitcoin Price')
plt.plot(predicted_stock_price, color = 'blue', label = ' Predicted Price')
plt.plot(predicted_stock_price_with_google_trends, color = 'green', label = 'Predicted Price with google trends')
plt.xticks(np.arange(0,len(test_set[60:,3]),50))
plt.title('Bitcoin Price Prediction')
plt.xlabel('Time')
plt.ylabel('Bitcoin Price')
plt.legend()
plt.show()
Output_Df = pd.DataFrame()
original_close_Df = merged_dataframe.iloc[int(n*0.7)+60:,4].reset_index(drop = True)
Output_Df['original_close_percentage'] = (original_close_Df-original_close_Df.shift(1))/original_close_Df*100
predicted_stock_price_ = pd.Series(predicted_stock_price.reshape(1,-1)[0])
Output_Df['model_close_percentage'] = (predicted_stock_price_-predicted_stock_price_.shift(1))/predicted_stock_price_*100
predicted_stock_price_with_google_trends_ = pd.Series(predicted_stock_price_with_google_trends.reshape(1,-1)[0])
Output_Df['model_with_google_trends_close_percentage'] = (predicted_stock_price_with_google_trends_-predicted_stock_price_with_google_trends_.shift(1))/predicted_stock_price_with_google_trends_*100
Output_Df.dropna(inplace=True)
Output_Df['ori_gain']=Output_Df['original_close_percentage']>0
Output_Df['no_google_trends_gain']=Output_Df['model_close_percentage']>0
Output_Df['google_trends_gain']=Output_Df['model_with_google_trends_close_percentage']>0
print((Output_Df['ori_gain'] == Output_Df['no_google_trends_gain']).mean())
print((Output_Df['ori_gain'] == Output_Df['google_trends_gain']).mean())
'''